### Notebook part 1

# Data Cleaning
_By **Avi Patel**_

## Overview
This project seeks to create a model that creates one line abstractive summary of a given news article. This model will help to get a quick idea on what the article is all about without having to read all whole news.


## Business Problem
News/Media/Blogging Companies always wants a catchy  headlines for the articles they post so that they can get maximum number of clicks. A machine learning algorithm can be  used to create a headline based on the context of the article.


In [ ]:
import numpy as np
import pandas as pd
import re
!pip install contractions
import contractions
from time import time
import spacy
spacy.prefer_gpu()
nlp = spacy.load('en', disable=['ner', 'parser']) # disabling Named Entity Recognition for speed

In [2]:
df1 = pd.read_csv('/content/drive/MyDrive/news_summary.csv', encoding='iso-8859-1')
df2 = pd.read_csv('/content/drive/MyDrive/news_summary_more.csv', encoding='iso-8859-1')

In [3]:
df1.head(2)

,author,date,headlines,read_more,text,ctext
0,Chhavi Tyagi,"03 Aug 2017,Thursday",Daman & Diu revokes mandatory Rakshabandhan in...,http://www.hindustantimes.com/india-news/raksh...,The Administration of Union Territory Daman an...,The Daman and Diu administration on Wednesday ...
1,Daisy Mowke,"03 Aug 2017,Thursday",Malaika slams user who trolled her for 'divorc...,http://www.hindustantimes.com/bollywood/malaik...,Malaika Arora slammed an Instagram user who tr...,"From her special numbers to TV?appearances, Bo..."


In [4]:
df2.head(2)

,headlines,text
0,upGrad learner switches to career in ML & Al w...,"Saurav Kant, an alumnus of upGrad and IIIT-B's..."
1,Delhi techie wins free food from Swiggy for on...,Kunal Shah's credit card bill payment platform...


In [5]:
pre1 = df1.copy()
pre2 = df2.copy()
pre1['text'] =  pre1['text'].str.cat(pre1['ctext'], sep = " ")

In [6]:
df = pd.DataFrame()
df['text'] = pd.concat([pre2['text'], pre1['text']], ignore_index=True)
df['summary'] = pd.concat([pre2['headlines'],pre1['headlines']],ignore_index = True)

In [7]:
df.shape

(102915, 2)

In [8]:
df.head()

,text,summary
0,"Saurav Kant, an alumnus of upGrad and IIIT-B's...",upGrad learner switches to career in ML & Al w...
1,Kunal Shah's credit card bill payment platform...,Delhi techie wins free food from Swiggy for on...
2,New Zealand defeated India by 8 wickets in the...,New Zealand end Rohit Sharma-led India's 12-ma...
3,"With Aegon Life iTerm Insurance plan, customer...",Aegon life iTerm insurance plan helps customer...
4,Speaking about the sexual harassment allegatio...,"Have known Hirani for yrs, what if MeToo claim..."


**Cleaning Text and Summary**

---



In [9]:
def text_clean(column):
    for row in column:
        
        #REGEX IN ORDER
        
        row=re.sub("(\\t)", ' ', str(row)).lower() #remove escape charecters
        row=re.sub("(\\r)", ' ', str(row)).lower() 
        row=re.sub("(\\n)", ' ', str(row)).lower()
        
        row=re.sub("(__+)", ' ', str(row)).lower()   #remove _ if it occors more than one time consecutively
        row=re.sub("(--+)", ' ', str(row)).lower()   #remove - if it occors more than one time consecutively
        row=re.sub("(~~+)", ' ', str(row)).lower()   #remove ~ if it occors more than one time consecutively
        row=re.sub("(\+\++)", ' ', str(row)).lower()   #remove + if it occors more than one time consecutively
        row=re.sub("(\.\.+)", ' ', str(row)).lower()   #remove . if it occors more than one time consecutively
        
        row=re.sub(r"[<>()|&©ø\[\]\'\",;?~*!]", ' ', str(row)).lower() #remove <>()|&©ø"',;?~*!
        
        row=re.sub("(mailto:)", ' ', str(row)).lower() #remove mailto:
        row=re.sub(r"(\\x9\d)", ' ', str(row)).lower() #remove \x9* in text
        row=re.sub("([iI][nN][cC]\d+)", 'INC_NUM', str(row)).lower() #replace INC nums to INC_NUM
        row=re.sub("([cC][mM]\d+)|([cC][hH][gG]\d+)", 'CM_NUM', str(row)).lower() #replace CM# and CHG# to CM_NUM
        
        
        row=re.sub("(\.\s+)", ' ', str(row)).lower() #remove full stop at end of words(not between)
        row=re.sub("(\-\s+)", ' ', str(row)).lower() #remove - at end of words(not between)
        row=re.sub("(\:\s+)", ' ', str(row)).lower() #remove : at end of words(not between)
        
        row=re.sub("(\s+.\s+)", ' ', str(row)).lower() #remove any single charecters hanging between 2 spaces
        
        #Replace any url as such https://abc.xyz.net/browse/sdf-5327 ====> abc.xyz.net
        try:
            url = re.search(r'((https*:\/*)([^\/\s]+))(.[^\s]+)', str(row))
            repl_url = url.group(3)
            row = re.sub(r'((https*:\/*)([^\/\s]+))(.[^\s]+)',repl_url, str(row))
        except:
            pass #there might be emails with no url in them
        
        row = re.sub("(\s+)",' ',str(row)).lower() #remove multiple spaces
        
        #Should always be last
        row=re.sub("(\s+.\s+)", ' ', str(row)).lower() #remove any single charecters hanging between 2 spaces

        yield row

In [10]:
cleaned_text = text_clean(df['text'])
cleaned_summary = text_clean(df['summary'])

In [11]:
t = time()

text = [str(doc) for doc in nlp.pipe(cleaned_text, batch_size=5000, n_threads=-1)]

print('Time to clean up everything: {} mins'.format(round((time() - t) / 60, 2)))

Time to clean up everything: 4.66 mins


In [12]:
t = time()

summary = [str(doc) for doc in nlp.pipe(cleaned_summary, batch_size=5000, n_threads=-1)]

print('Time to clean up everything: {} mins'.format(round((time() - t) / 60, 2)))

Time to clean up everything: 0.89 mins


In [13]:
df['cleaned_text'] = pd.Series(text)
df['cleaned_summary'] = pd.Series(summary)


**Contractions to Expansion**

---



In [14]:
df.cleaned_summary = df.cleaned_summary.apply(lambda x: contractions.fix(x))

In [15]:
df.cleaned_text = df.cleaned_text.apply(lambda x: contractions.fix(x))

In [16]:
df.head()

,text,summary,cleaned_text,cleaned_summary
0,"Saurav Kant, an alumnus of upGrad and IIIT-B's...",upGrad learner switches to career in ML & Al w...,saurav kant an alumnus of upgrad and iiit-b pg...,upgrad learner switches to career in ml al wit...
1,Kunal Shah's credit card bill payment platform...,Delhi techie wins free food from Swiggy for on...,kunal shah credit card bill payment platform c...,delhi techie wins free food from swiggy for on...
2,New Zealand defeated India by 8 wickets in the...,New Zealand end Rohit Sharma-led India's 12-ma...,new zealand defeated india by wickets in the f...,new zealand end rohit sharma-led india 12-matc...
3,"With Aegon Life iTerm Insurance plan, customer...",Aegon life iTerm insurance plan helps customer...,with aegon life iterm insurance plan customers...,aegon life iterm insurance plan helps customer...
4,Speaking about the sexual harassment allegatio...,"Have known Hirani for yrs, what if MeToo claim...",speaking about the sexual harassment allegatio...,have known hirani for yrs what if metoo claims...


**Save Cleaned DataFrame**

---



In [17]:
#df.to_csv('cleaned_news_summary.csv', index=False)

**Next Step**


---



The Next Step Involves modeling of the data using Seq2Se1 LSTM which can be found in (LSTM.ipynb)